In [1]:
#https://keras.io/examples/timeseries/timeseries_transformer_classification/
logger_name = "lstnet"

# Path appended in order to import from util
import sys
sys.path.append('..')
from util.model_util import LoadModel, SaveModel, SaveResults, SaveHistory
from util.Msglog import LogInit

from datetime import datetime

#from lstnet_util import GetArguments, LSTNetInit
from pandas_util import DataUtil
#from lstnet_model import PreSkipTrans, PostSkipTrans, PreARTrans, PostARTrans, LSTNetModel, ModelCompile
#from lstnet_plot import AutoCorrelationPlot, PlotHistory, PlotPrediction

import tensorflow as tf
import argparse

import numpy as np
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras import backend as K


In [2]:
# Reading data
filename = 'D:\\Work\\GitHub\\network_performance_anomaly_detect\\data\\bq-results-train.csv'
trainpercent = 0.6
validpercent = 0.2
horizon=0
window=10
normalize=0
Data = DataUtil(filename,
                trainpercent,
                validpercent,
                horizon,
                window,
                normalise=normalize,
                dependent_variable="iperf_throughput_1_thread")


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,14,20,45,58,59,82,83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


8042.861648594496
5088.83680043049


In [3]:
#Data.train[0].shape
print(Data.train[0])
#type(Data.train)
#print(len(Data.train[0]))
#Data.train
#n_classes = len(np.unique(Data.train))

[[[ 0.05524609 -1.0570084   0.52944064]
  [ 0.05917626 -1.0566899   0.52944064]
  [ 0.04876131 -1.0565562   0.52944064]
  ...
  [ 0.09808496 -1.0573269   0.52944064]
  [ 0.18454872 -1.0617254   0.52944064]
  [-0.01471096 -1.0613348   0.52944064]]

 [[ 0.05917626 -1.0566899   0.52944064]
  [ 0.04876131 -1.0565562   0.52944064]
  [-0.14597867 -1.055323    0.52944064]
  ...
  [ 0.18454872 -1.0617254   0.52944064]
  [-0.01471096 -1.0613348   0.52944064]
  [ 0.72219616 -1.0669048   0.52944064]]

 [[ 0.04876131 -1.0565562   0.52944064]
  [-0.14597867 -1.055323    0.52944064]
  [-0.03554086 -1.057255    0.52944064]
  ...
  [-0.01471096 -1.0613348   0.52944064]
  [ 0.72219616 -1.0669048   0.52944064]
  [-0.15403552 -1.0576661   0.52944064]]

 ...

 [[ 1.1745589  -1.1839039  -1.888774  ]
  [ 1.7615299  -1.1840478  -1.888774  ]
  [ 1.8244127  -1.1839862  -1.888774  ]
  ...
  [ 2.155333   -1.1845     -1.888774  ]
  [ 2.6552508  -1.1852913  -1.888774  ]
  [ 3.3866558  -1.1849214  -1.888774  ]]

 [

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [6]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [7]:
print(f"data.train: {Data.train}")

data.train: [array([[[ 0.05524609, -1.0570084 ,  0.52944064],
        [ 0.05917626, -1.0566899 ,  0.52944064],
        [ 0.04876131, -1.0565562 ,  0.52944064],
        ...,
        [ 0.09808496, -1.0573269 ,  0.52944064],
        [ 0.18454872, -1.0617254 ,  0.52944064],
        [-0.01471096, -1.0613348 ,  0.52944064]],

       [[ 0.05917626, -1.0566899 ,  0.52944064],
        [ 0.04876131, -1.0565562 ,  0.52944064],
        [-0.14597867, -1.055323  ,  0.52944064],
        ...,
        [ 0.18454872, -1.0617254 ,  0.52944064],
        [-0.01471096, -1.0613348 ,  0.52944064],
        [ 0.72219616, -1.0669048 ,  0.52944064]],

       [[ 0.04876131, -1.0565562 ,  0.52944064],
        [-0.14597867, -1.055323  ,  0.52944064],
        [-0.03554086, -1.057255  ,  0.52944064],
        ...,
        [-0.01471096, -1.0613348 ,  0.52944064],
        [ 0.72219616, -1.0669048 ,  0.52944064],
        [-0.15403552, -1.0576661 ,  0.52944064]],

       ...,

       [[ 1.1745589 , -1.1839039 , -1.888774  ]

In [9]:
input_shape = Data.train[0].shape[1:]
#print(f"data.train: {type(Data.train)}")
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    Data.train[0],
    Data.train[1],
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

NameError: name 'n_classes' is not defined

In [ ]:
#training = np.array([np.array(xi) for xi in Data.train])
latency = []
throughput = []
tcp_buffer_size = []
counter = 0
for x in range(1):
    
    for y in range(10):
        latency_list = []
        throughput_list = []
        tcp_buffer_size_list = []
        for z in Data.train[x][y]:
            #print(f"y is: {z}")
            latency_list.append(z[0])
            throughput_list.append(z[1])
            tcp_buffer_size_list.append(z[2])
        latency.append(latency_list)
        throughput.append(throughput_list)
        tcp_buffer_size.append(tcp_buffer_size_list)
    break
#print(f"latency: {latency[0]}")
training = np.array([np.array(xi) for xi in latency])
training = Data.train.reshape((Data.train[0].shape[0], data.train[0].shape[1], 1))
#x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

#print(f"data train is: {training}")

In [ ]:
val_performance = {}
performance = {}

In [ ]:
#Root relative squared error
def tf_rse(y_true, y_pred):
    #
    # The formula is:
    #           K.sqrt(K.sum(K.square(y_true - y_pred)))     
    #    RSE = -----------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)))       
    #
    #           K.sqrt(K.sum(K.square(y_true - y_pred))/(N-1))
    #        = ----------------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)/(N-1)))
    #
    #
    #           K.sqrt(K.mean(K.square(y_true - y_pred)))
    #        = ------------------------------------------
    #           K.std(y_true)
    #
    num = K.sqrt(K.mean(K.square(y_true - y_pred), axis=None))
    den = K.std(y_true, axis=None)
    
    return num / den

def rse_test1(y_true, y_pred):
    return K.square(y_true - y_pred)

def rse_test2(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred), axis=None))

def rse_test3(y_true, y_pred):
    return K.std(y_true, axis=None)

def rse(y_true, y_pred):
    #
    # The formula is:
    #           K.sqrt(K.sum(K.square(y_true - y_pred)))     
    #    RSE = -----------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)))       
    #
    #           K.sqrt(K.sum(K.square( y_true - y_pred))/(N-1))
    #        = ----------------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)/(N-1)))
    #
    #
    #           K.sqrt(K.mean(K.square(y_true - y_pred)))
    #        = ------------------------------------------
    #           K.std(y_true)
    #
    num = np.sqrt(np.mean(np.square(y_true - y_pred), axis=None))
    den = np.std(y_true, axis=None)
    
    return num / den

def tf_corr(y_true, y_pred):
    #
    # This function calculates the correlation between the true and the predicted outputs
    #
    num1 = y_true - K.mean(y_true, axis=0)
    num2 = y_pred - K.mean(y_pred, axis=0)
    
    num  = K.mean(num1 * num2, axis=0)
    den  = K.std(y_true, axis=0) * K.std(y_pred, axis=0)
    
    return K.mean(num / den)

def corr(y_true, y_pred):
    #
    # This function calculates the correlation between the true and the predicted outputs
    #
    num1 = y_true - np.mean(y_true, axis=0)
    num2 = y_pred - np.mean(y_pred, axis=0)
    
    num  = np.mean(num1 * num2, axis=0)
    den  = np.std(y_true, axis=0) * np.std(y_pred, axis=0)
    
    return np.mean(num / den)

def single_rse(y_true, y_pred):
    #
    # The formula is:
    #           K.sqrt(K.sum(K.square(y_true - y_pred)))     
    #    RSE = -----------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)))       
    #
    #           K.sqrt(K.sum(K.square(y_true - y_pred))/(N-1))
    #        = ----------------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)/(N-1)))
    #
    #
    #           K.sqrt(K.mean(K.square(y_true - y_pred)))
    #        = ------------------------------------------
    #           K.std(y_true)
    #
    num = K.sqrt(K.mean(K.square(y_true[:,0] - y_pred[:,0]), axis=None))
    den = K.std(y_true, axis=None)
    
    return num / den


def single_corr(y_true, y_pred):
    #
    # This function calculates the correlation between the true and the predicted outputs
    #
    num1 = y_true[:,0] - K.mean(y_true[:,0], axis=0)
    num2 = y_pred[:,0] - K.mean(y_pred[:,0], axis=0)
    
    num  = K.mean(num1 * num2, axis=0)
    den  = K.std(y_true[:,0], axis=0) * K.std(y_pred[:,0], axis=0)
    
    return K.mean(num / den)

def mean_absolute_percentage_error(y_true, y_pred): 
  return np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=10,
                                                mode='min')

In [ ]:
Data.train[0][:,:,0:3][0]

In [ ]:
Data.train[1][:,0]

In [ ]:
accuracy_stats = {}
accuracy_stats['rmse'] = {}
accuracy_stats['rse'] = {}
accuracy_stats['corr'] = {}
accuracy_stats['accuracy'] = {}
accuracy_stats['mae'] = {}
accuracy_stats['predicted'] = {}
accuracy_stats['mse'] = {}

## Simple MLP

Here we are testing the effectiveness of a simple ANN (MLP) on our dataset

In [ ]:
linear = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(10,3)),
    tf.keras.layers.Dense(units=15),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=15),
    tf.keras.layers.Dense(units=1),
])

In [ ]:
linear.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf_rse, tf_corr, tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError(), tf.metrics.RootMeanSquaredError()])

In [ ]:
Data.train[1][:,0].shape[0]

In [ ]:
linear.fit(Data.train[0][:,:,:], Data.train[1][:,0].reshape(Data.train[1][:,0].shape[0],1), epochs=100, validation_data=(Data.valid[0][:,:,:], Data.valid[1][:,0].reshape(Data.valid[1][:,0].shape[0],1)), callbacks=[early_stopping])

In [ ]:
#linear_stats = linear.evaluate(Data.test[0], Data.test[1][:,0].reshape(2933,1))
linear_stats = linear.evaluate(Data.test[0], Data.test[1][:,0].reshape(25230,1))

linear_stats

In [ ]:
linear_predicted = linear.predict(Data.test[0][:,:,:])

np.save('predicted_results/mlp_predicted.npy', linear_predicted)
np.savetxt('predicted_results/mlp_predicted.txt', linear_predicted)
linear.save_weights('./checkpoints/linear_weights')

In [ ]:
from scipy.stats import iqr
import matplotlib.cm as cm
Data.test[1][0:25230,0]
linear_predicted[0:25230]
inquartileRange = iqr(Data.test[1][0:25230,0],interpolation = 'midpoint')
print(inquartileRange)
higherRange = inquartileRange * 1.5

import numpy as np
# First quartile (Q1)
#print(Data.test[])
Q1 = np.median(Data.test[1][:12615,0])
  
# Third quartile (Q3)
Q3 = np.median(Data.test[1][12615:,0])
  
# Interquartile range (IQR)
IQR = Q3 - Q1
print(f"IQR: {IQR}")
# Quartile Deviation
qd = IQR / 2
  
print(f"Quartile Deviation: {qd}")
highRange = IQR + qd
lowRange = IQR - qd
print(f"High Range: {highRange}")
print(f"Low Range: {lowRange}")
# plt.plot(Data.test[1][0:25230,0])
# print(linear_predicted[0:25230])
# anomolyList =[]
# anomolyListXCoords =[]
# count = 0
# for x in Data.test[1][0:25230,0]:
#     if x > 2 or x <-1:
#         anomolyList.append([x])
#         anomolyListXCoords.append(count)
#     count = count + 1
# colors = cm.rainbow(np.linspace(1, 1, 2119))
# #print(anomolyList)
# #plt.scatter(linear_predicted[0:25230])
# plt.scatter(anomolyListXCoords, anomolyList, c=colors)
# plt.legend(["Actual", "Anomoly Points"])
# plt.show()
print(Data.test[0][1,:])
#Data.test[x] shows data in the format [throughput, latency, bufferSize] . Focus Anomoly detection on both Throughput and Latency

In [ ]:
import scipy
import matplotlib.cm as cm
dataRange = 500
Data.test[1][0:25230,0]
linear_predicted[0:25230]
plt.plot(Data.test[1][0:dataRange,0])
#print(linear_predicted[0:25230])
anomalyList =[]
anomalyListXCoords =[]
count = 0
total=0
for x in range(dataRange):
    difference = abs(Data.test[1][x,0]-linear_predicted[x])
    total = difference + total
    count += 1
total = total / count
print(f"The Average is: {total}")
count = 0
for x in range(dataRange):
    difference = abs(Data.test[1][x,0]-linear_predicted[x])
    if difference > 3*total:
        anomalyList.append(Data.test[1][x,0])
        anomalyListXCoords.append(count)
    count += 1
percentageAnomalous =  round(len(anomalyList)/dataRange *100)
print(f"The percentage of total anomalous points was: {percentageAnomalous}%")
colors = cm.rainbow(np.linspace(1, 1, len(anomalyList)))
print(f"The len of anomalyList: {len(anomalyList)}")
#print(anomolyList)
#plt.scatter(linear_predicted[0:25230])
#print(f"The anomalyList: {anomalyList}\nThe anomalyListXCoords: {anomalyListXCoords}")

plt.scatter(anomalyListXCoords, anomalyList, c=colors)
#plt.scatter([1762, 4443, 19902, 23501, 25112], [3.7193446, 4.0872087, 4.4067316, 3.9032767, 4.3992643])#, c=colors)

plt.legend(["Actual", "Anomoly Points"])
plt.show()

In [ ]:
plt.plot(Data.test[1][0:25230,0])
plt.plot(linear_predicted[0:25230])
plt.legend(["Actual", "Predicted"])
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

mse = mean_squared_error(Data.test[1][:,0], linear_predicted.flatten())
rmse = sqrt(mean_squared_error(Data.test[1][:,0], linear_predicted.flatten()))
mae = mean_absolute_error(Data.test[1][:,0], linear_predicted.flatten())
rse_val = rse(Data.test[1][:,0], linear_predicted.flatten())

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'RSE: {rse_val}')

accuracy_stats['rmse']['mlp'] = rmse
accuracy_stats['rse']['mlp'] = rse_val
# accuracy_stats['corr']['mlp'] = corr
# accuracy_stats['accuracy']['mlp'] = acc
accuracy_stats['mae']['mlp'] = mae
accuracy_stats['predicted']['mlp'] = linear_predicted
accuracy_stats['mse']['mlp'] = mse

## SIMPLE LSTM

Here we run our data through a simple lstm for comparison

In [ ]:
Data.train[0][:,:,0][0]

In [ ]:
Data.train[0][:,:,0][0]

In [ ]:
# define lstm network

# lstm_model = tf.keras.models.Sequential([
#     # Shape [batch, time, features] => [batch, time, lstm_units]
#     tf.keras.layers.GRU(32, input_shape=(10), return_sequences=False),
#     # Shape => [batch, time, features]
#     tf.keras.layers.Dense(units=1)
# ])

ts_inputs = tf.keras.Input(shape=(10,3))
x = tf.keras.layers.LSTM(units=250, dropout=0.1, recurrent_dropout=0.1)(ts_inputs)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1,activation='linear')(x)
lstm_model = tf.keras.Model(inputs=ts_inputs, outputs=outputs)

In [ ]:
lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf_rse, tf_corr, tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError(), tf.metrics.RootMeanSquaredError()])

In [ ]:
lstm_model.summary()

In [ ]:
history = lstm_model.fit(Data.train[0][:,:,:], Data.train[1][:,0], 
               epochs=100, 
               validation_data=(Data.valid[0][:,:,:], Data.valid[1][:,0]), 
               callbacks=[early_stopping],
               shuffle=False)

In [ ]:
lstm_stats = lstm_model.evaluate(Data.test[0][:,:,:], Data.test[1][:,0])
lstm_stats

In [ ]:
lstm_predicted = lstm_model.predict(Data.test[0][:,:,:]).flatten()

np.save('predicted_results/lstm_predicted.npy', lstm_predicted)
np.savetxt('predicted_results/lstm_predicted.txt', lstm_predicted)
lstm_model.save_weights('./checkpoints/lstm_weights')

In [ ]:
lstm_predicted = np.load('predicted_results/lstm_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]
data_true = Data.test[1][:,0]*Data.normalize_std[0] + Data.normalize_mean[0]
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100
lstm_mape = mean_absolute_percentage_error(data_true, lstm_predicted)
lstm_mape

In [ ]:
plt.plot(Data.test[1][0:50,0])
plt.plot(lstm_predicted[0:50])
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(25, 10))
plt.plot(Data.test[1][:,0],label="actual")
plt.plot(lstm_predicted, color='r',label="predicted")
plt.legend(loc='best', fontsize='xx-large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

mse = mean_squared_error(Data.test[1][:,0], lstm_predicted)
rmse = sqrt(mean_squared_error(Data.test[1][:,0], lstm_predicted))
mae = mean_absolute_error(Data.test[1][:,0], lstm_predicted)
rse_val = rse(Data.test[1][:,0], lstm_predicted)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'RSE: {rse_val}')

accuracy_stats['rmse']['lstm'] = rmse
accuracy_stats['rse']['lstm'] = rse_val
# accuracy_stats['corr']['mlp'] = corr
# accuracy_stats['accuracy']['mlp'] = acc
accuracy_stats['mae']['lstm'] = mae
accuracy_stats['predicted']['lstm'] = lstm_predicted
accuracy_stats['mse']['lstm'] = mse

In [ ]:
lstm_predicted.shape

In [ ]:
Data.test[1][:,0].shape

## Simple GRU

Do the same thing but with a GRU


In [ ]:
ts_inputs = tf.keras.Input(shape=(10,3))
x = tf.keras.layers.GRU(units=250, dropout=0.1)(ts_inputs)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1,activation='linear')(x)
gru_model = tf.keras.Model(inputs=ts_inputs, outputs=outputs)

In [ ]:
gru_model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf_rse, tf_corr, tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError(), tf.metrics.RootMeanSquaredError()])

In [ ]:
gru_model.summary()

In [ ]:
gru_model.fit(Data.train[0][:,:,:], Data.train[1][:,0], 
               epochs=100, 
               validation_data=(Data.valid[0][:,:,:], Data.valid[1][:,0]), 
               callbacks=[early_stopping],
               shuffle=True)

In [ ]:
gru_model.evaluate(Data.test[0][:,:,:], Data.test[1][:,0])

In [ ]:
gru_predicted = gru_model.predict(Data.test[0][:,:,:]).flatten()

np.save('predicted_results/gru_predicted.npy', gru_predicted)
np.savetxt('predicted_results/gru_predicted.txt', gru_predicted)
gru_model.save_weights('./checkpoints/gru_weights')

In [ ]:
gru_predicted = np.load('predicted_results/gru_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]
data_true = Data.test[1][:,0]*Data.normalize_std[0] + Data.normalize_mean[0]
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100
gru_mape = mean_absolute_percentage_error(data_true, gru_predicted)
gru_mape

In [ ]:
plt.plot(Data.test[1][0:100,0], label='actual')
plt.plot(gru_predicted[0:100], label="predicted")
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

mse = mean_squared_error(Data.test[1][:,0], gru_predicted)
rmse = sqrt(mean_squared_error(Data.test[1][:,0], gru_predicted))
mae = mean_absolute_error(Data.test[1][:,0], gru_predicted)
rse_val = rse(Data.test[1][:,0], gru_predicted)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'RSE: {rse_val}')

accuracy_stats['rmse']['gru'] = rmse
accuracy_stats['rse']['gru'] = rse_val
# accuracy_stats['corr']['mlp'] = corr
# accuracy_stats['accuracy']['mlp'] = acc
accuracy_stats['mae']['gru'] = mae
accuracy_stats['predicted']['gru'] = gru_predicted
accuracy_stats['mse']['gru'] = mse

### CNN MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling1D


# cnn_model = Sequential()
# cnn_model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(10, 3)))
# cnn_model.add(MaxPooling1D(pool_size=2))
# cnn_model.add(Flatten())
# cnn_model.add(Dense(50, activation='relu'))
# cnn_model.add(Dense(1))
# cnn_model.compile(optimizer='adam', loss='mse')

cnn_model = Sequential()
cnn_model.add(Conv1D(filters=30, kernel_size=5, activation='relu', padding='SAME', strides=1, input_shape=(10, 3)))
cnn_model.add(MaxPooling1D(pool_size=2, strides=2))
cnn_model.add(Conv1D(filters=45, kernel_size=5, activation='relu', padding='SAME', strides=1))
cnn_model.add(MaxPooling1D(pool_size=2, strides=2))
cnn_model.add(Conv1D(filters=60, kernel_size=5, activation='relu', padding='SAME', strides=1))
cnn_model.add(MaxPooling1D(pool_size=2, strides=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation='relu'))
cnn_model.add(Dense(1))
# cnn_model.compile(optimizer='adam', loss='mse')

In [ ]:
cnn_model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf_rse, tf_corr, tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError(), tf.metrics.RootMeanSquaredError()])

In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.fit(Data.train[0][:,:,:], Data.train[1][:,0], 
               epochs=100, 
               validation_data=(Data.valid[0][:,:,:], Data.valid[1][:,0]), 
               callbacks=[early_stopping],
               shuffle=True)

In [ ]:
cnn_stats = cnn_model.evaluate(Data.test[0][:,:,:], Data.test[1][:,0])
cnn_stats

In [ ]:
cnn_predicted = cnn_model.predict(Data.test[0][:,:,:]).flatten()

np.save('predicted_results/cnn_predicted.npy', cnn_predicted)
np.savetxt('predicted_results/cnn_predicted.txt', cnn_predicted)
cnn_model.save_weights('./checkpoints/cnn_weights')

In [ ]:
plt.plot(Data.test[1][0:100,0], label='actual')
plt.plot(cnn_predicted[0:100], label="predicted")
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

mse = mean_squared_error(Data.test[1][:,0], cnn_predicted)
rmse = sqrt(mean_squared_error(Data.test[1][:,0], cnn_predicted))
mae = mean_absolute_error(Data.test[1][:,0], cnn_predicted)
rse_val = rse(Data.test[1][:,0], cnn_predicted)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'RSE: {rse_val}')

accuracy_stats['rmse']['cnn'] = rmse
accuracy_stats['rse']['cnn'] = rse_val
# accuracy_stats['corr']['mlp'] = corr
# accuracy_stats['accuracy']['mlp'] = acc
accuracy_stats['mae']['cnn'] = mae
accuracy_stats['predicted']['cnn'] = cnn_predicted
accuracy_stats['mse']['cnn'] = mse

In [ ]:
cnn_predicted = np.load('predicted_results/cnn_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]
data_true = Data.test[1][:,0]*Data.normalize_std[0] + Data.normalize_mean[0]

cnn_mape = mean_absolute_percentage_error(data_true, cnn_predicted)
cnn_mape

### CNN-LSTM HYBRID MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, Conv2D, LSTM, MaxPooling1D

In [ ]:
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='SAME', strides=1, input_shape=(10, 3)))
cnn_lstm_model.add(MaxPooling1D(pool_size=2, strides=2))
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='SAME', strides=1))
cnn_lstm_model.add(MaxPooling1D(pool_size=2, strides=2))
# cnn_lstm_model.add(Conv1D(filters=10, kernel_size=3, activation='relu', padding='SAME', strides=1))
# cnn_lstm_model.add(MaxPooling1D(pool_size=2, strides=2))
# cnn_model.add(Flatten())
cnn_lstm_model.add(LSTM(32, dropout=0.1, recurrent_dropout=0.1, return_sequences=False))
cnn_lstm_model.add(Dense(100, activation='relu'))
cnn_lstm_model.add(Dense(1))

In [ ]:
cnn_lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                       optimizer=tf.optimizers.Adam(),
                       metrics=[tf_rse, tf_corr, tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError(), tf.metrics.RootMeanSquaredError()])

In [ ]:
cnn_lstm_model.summary()

In [ ]:
cnn_lstm_model.fit(Data.train[0][:,:,:], Data.train[1][:,0], 
                   epochs=100, 
                   validation_data=(Data.valid[0][:,:,:], Data.valid[1][:,0]), 
                   callbacks=[early_stopping],
                   shuffle=True)

In [ ]:
cnn_lstm1_stats = cnn_lstm_model.evaluate(Data.test[0][:,:,:], Data.test[1][:,0])
cnn_lstm1_stats

In [ ]:
cnn_lstm_predicted = cnn_lstm_model.predict(Data.test[0][:,:,:]).flatten()

In [ ]:
plt.plot(Data.test[1][0:100,0], label='actual')
plt.plot(cnn_lstm_predicted[0:100], label="predicted")
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

mse = mean_squared_error(Data.test[1][:,0], cnn_lstm_predicted)
rmse = sqrt(mean_squared_error(Data.test[1][:,0], cnn_lstm_predicted))
mae = mean_absolute_error(Data.test[1][:,0], cnn_lstm_predicted)
rse_val = rse(Data.test[1][:,0], cnn_lstm_predicted)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'RSE: {rse_val}')

accuracy_stats['rmse']['cnn_lstm'] = rmse
accuracy_stats['rse']['cnn_lstm'] = rse_val
# accuracy_stats['corr']['mlp'] = corr
# accuracy_stats['accuracy']['mlp'] = acc
accuracy_stats['mae']['cnn_lstm'] = mae
accuracy_stats['predicted']['cnn_lstm'] = cnn_lstm_predicted
accuracy_stats['mse']['cnn_lstm'] = mse

### Take 2

In [ ]:
Data.train[0].shape

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=10,
                                                mode='min')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, LSTM, MaxPooling2D, Reshape, TimeDistributed, Input, Dropout, GRU

In [ ]:
X = Input(shape = (10,3))


#CNN
C = Reshape((10, 3, 1))(X)

# Apply a Conv2D that will transform it into data of dimensions (batchsize, time, 1, NumofFilters)
C = Conv2D(filters=200, kernel_size=(1, 3), kernel_initializer='glorot_uniform')(C)
C = Dropout(0.2)(C)

# Adjust data dimensions by removing axis=2 which is always equal to 1
c_shape = K.int_shape(C)
C = Reshape((c_shape[1], c_shape[3]))(C)

# Apply a GRU layer (with activation set to 'relu' as per the paper) and take the returned states as result
_, R = GRU(200, activation="relu", return_sequences = False, return_state = True)(C)
R    = Dropout(0.2)(R)
Y = Flatten()(R)
Y = Dense(3)(Y)
cnn_lstm_model2 = Model(inputs = X, outputs = Y)

In [ ]:
cnn_lstm_model2.compile(loss=tf.losses.MeanSquaredError(),
                       optimizer=tf.optimizers.Adam(),
                       metrics=[tf_rse, tf_corr, tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError(), tf.metrics.RootMeanSquaredError()])

In [ ]:
cnn_lstm_model2.summary()

In [ ]:
# tf.random.set_seed(123456)
cnn_lstm_model2.fit(Data.train[0][:,:,:], Data.train[1][:,0], 
                   epochs=10, 
                   validation_data=(Data.valid[0][:,:,:], Data.valid[1]), 
                   callbacks=[early_stopping],
                   shuffle=True)

In [ ]:
cnn_lstm_model2.evaluate(Data.test[0][:,:,:], Data.test[1])

In [ ]:
cnn_lstm_model2_predicted

In [ ]:
cnn_lstm_model2_predicted = cnn_lstm_model2.predict(Data.test[0][:,:,:])

np.save('predicted_results/cnn_lstm_predicted.npy', cnn_lstm_model2_predicted)
np.savetxt('predicted_results/cnn_lstm_predicted.txt', cnn_lstm_model2_predicted)
cnn_lstm_model.save_weights('./checkpoints/cnn_lstm_weights')

In [ ]:
plt.plot(Data.test[1][0:100,0], label='actual')
plt.plot(cnn_lstm_model2_predicted[0:100,0], label="predicted")
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

mse = mean_squared_error(Data.test[1][:,0], cnn_lstm_model2_predicted.flatten())
rmse = sqrt(mean_squared_error(Data.test[1][:,0], cnn_lstm_model2_predicted.flatten()))
mae = mean_absolute_error(Data.test[1][:,0], cnn_lstm_model2_predicted.flatten())
rse_val = rse(Data.test[1][:,0], cnn_lstm_model2_predicted[:,0].flatten())

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'RSE: {rse_val}')

accuracy_stats['rmse']['cnn_lstm2'] = rmse
accuracy_stats['rse']['cnn_lstm2'] = rse_val
# accuracy_stats['corr']['mlp'] = corr
# accuracy_stats['accuracy']['mlp'] = acc
accuracy_stats['mae']['cnn_lstm2'] = mae
accuracy_stats['predicted']['cnn_lstm2'] = cnn_lstm_model2_predicted
accuracy_stats['mse']['cnn_lstm2'] = mse

### LSTNET MODEL

In [ ]:
from tensorflow.keras.models import Model, model_from_json

custom_objects = {
        'PreSkipTrans': PreSkipTrans,
        'PostSkipTrans': PostSkipTrans,
        'PreARTrans': PreARTrans,
        'PostARTrans': PostARTrans
        }


def custom_loss(y_true, y_pred):
    # print("CUSTOM LOSS BABBBY")
    # print(type(y_true))
    # print(type(y_pred))
    # print(y_true)
    # print(y_true.shape)
    # tf.print(y_true[:,0, sys.stdout)
    return tf.keras.losses.mean_absolute_error(y_true[:,0], y_pred[:,0])

file = 'models/model2.json'
lstnet_model = None
with open(file, "r") as json_file:
  lstnet_model = model_from_json(json_file.read(), custom_objects=custom_objects)


In [ ]:
lstnet_model.compile(loss=custom_loss,
                       optimizer=tf.optimizers.Adam(),
                       metrics=[single_rse, single_corr, tf_rse, tf_corr, tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError(), tf.metrics.RootMeanSquaredError()])

In [ ]:
lstnet_model.fit(Data.train[0][:,:,:], Data.train[1][:,0], 
                   epochs=20, 
                   validation_data=(Data.valid[0][:,:,:], Data.valid[1][:,0]), 
                   callbacks=[early_stopping],
                   shuffle=True,
                   batch_size=91)

In [ ]:
lstnet_model.summary()

In [ ]:
lstnet_model.evaluate(Data.test[0], Data.test[1])

In [ ]:
lstnet_predicted = lstnet_model.predict(Data.test[0])

np.save('predicted_results/lstnet_predicted.npy', lstnet_predicted)
np.savetxt('predicted_results/lstnet_predicted.txt', lstnet_predicted)
lstnet_model.save_weights('./checkpoints/lstnet_weights')

In [ ]:
plt.plot(Data.test[1][0:100,0], label='actual')
plt.plot(lstnet_predicted[0:100, 0], label="predicted")
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
print(f'MSE: {mean_squared_error(Data.test[1][:,0], lstnet_predicted[:,0])}')
print(f'RMSE: {sqrt(mean_squared_error(Data.test[1][:,0], lstnet_predicted[:,0]))}')
print(f'MAE: {mean_absolute_error(Data.test[1][:,0], lstnet_predicted[:,0])}')
print(f'RSE: {rse(Data.test[1][:,0], lstnet_predicted[:,0])}')

In [ ]:
Data.test[1][:,0]

In [ ]:
lstnet_predicted[:,0]

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100

In [ ]:
mean_absolute_percentage_error(Data.test[1][:,0]*Data.normalize_std[0] + Data.normalize_mean[0], lstnet_predicted[:,0]*Data.normalize_std[0] + Data.normalize_mean[0])

# Comparison between Different Methods

In [ ]:
mlp_predicted = np.load('predicted_results/mlp_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]
lstm_predicted = np.load('predicted_results/lstm_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]
gru_predicted = np.load('predicted_results/gru_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]
cnn_predicted = np.load('predicted_results/cnn_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]
cnn_lstm_predicted = np.load('predicted_results/cnn_lstm_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]
lstnet_predicted = np.load('predicted_results/lstnet_predicted.npy')*Data.normalize_std[0] + Data.normalize_mean[0]

data_true = Data.test[1][:,0]*Data.normalize_std[0] + Data.normalize_mean[0]

In [ ]:
data_true.shape

In [ ]:
from cycler import cycler

# Create cycler object. Use any styling from above you please
monochrome = (cycler('color', ['k']) * cycler('linestyle', ['-', '--', ':']) * cycler('marker', ['^',',', '.']))

# Print examples of output from cycler object. 
# A cycler object, when called, returns a `iter.cycle` object that iterates over items indefinitely

default_cycler = (cycler(color=['black', 'r', 'b', 'y', 'teal', 'green', 'purple']) +
                  cycler(linestyle=['-', '--', ':', '-.', '-.', '-', '--']))

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

cmap_1 = ListedColormap(["darkorange", "gold", "lawngreen", "lightseagreen"])

In [ ]:
start_time = 150
end_time = 200
plt.rcParams["figure.figsize"] = (10,7)
# plt.rc('axes', prop_cycle=default_cycler)
# plt.style.use('grayscale')
plt.plot(data_true[start_time:end_time], label='actual')
plt.plot(mlp_predicted[start_time:end_time, 0], label="mlp")
plt.plot(lstm_predicted[start_time:end_time], label="lstm")
plt.plot(gru_predicted[start_time:end_time], label="gru")
plt.plot(cnn_predicted[start_time:end_time], label="cnn")
plt.plot(cnn_lstm_predicted[start_time:end_time, 0], label="cnn+gru")
plt.plot(lstnet_predicted[start_time:end_time, 0], label="lstnet")
plt.ylabel('Throughput (Mbits/sec)')
plt.xlabel('Day')
plt.legend(loc='best')
plt.show()

### RMSE

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
mlp_rmse = sqrt(mean_squared_error(data_true, mlp_predicted[:, 0]))
mlp_rmse

In [ ]:
lstm_rmse = sqrt(mean_squared_error(data_true, lstm_predicted))
lstm_rmse

In [ ]:
gru_rmse = sqrt(mean_squared_error(data_true, gru_predicted))
gru_rmse

In [ ]:
cnn_rmse = sqrt(mean_squared_error(data_true, cnn_predicted))
cnn_rmse

In [ ]:
cnn_lstm_rmse = sqrt(mean_squared_error(data_true, cnn_lstm_predicted[:, 0]))
cnn_lstm_rmse

In [ ]:
lstnet_rmse = sqrt(mean_squared_error(data_true, lstnet_predicted[:, 0]))
lstnet_rmse

In [ ]:
fig = plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.bar(x=['ARIMA', 'MLP', 'LSTM', 'GRU', 'CNN', 'CNN+LSTM', 'LSTNet'], 
        height=[0.45, mlp_rmse, lstm_rmse, gru_rmse, cnn_rmse, cnn_lstm_rmse, lstnet_rmse])
ax.set_xlabel('Model')

ax.set_ylabel('Root Mean Squared Error (RMSE)')

### MAPE

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100

In [ ]:
mlp_mape = mean_absolute_percentage_error(data_true, mlp_predicted[:, 0])
mlp_mape

In [ ]:
lstm_mape = mean_absolute_percentage_error(data_true, lstm_predicted)
lstm_mape

In [ ]:
gru_mape = mean_absolute_percentage_error(data_true, gru_predicted)
gru_mape

In [ ]:
cnn_mape = mean_absolute_percentage_error(data_true, cnn_predicted)
cnn_mape

In [ ]:
cnn_lstm_mape = mean_absolute_percentage_error(data_true, cnn_lstm_predicted[:, 0])
cnn_lstm_mape

In [ ]:
lstnet_mape = mean_absolute_percentage_error(data_true, lstnet_predicted[:, 0])
lstnet_mape

In [ ]:
fig = plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.bar(x=['ARIMA', 'MLP', 'LSTM', 'GRU', 'CNN', 'CNN+LSTM', 'LSTNet'], 
        height=[79, mlp_mape, lstm_mape, gru_mape, cnn_mape, cnn_lstm_mape, lstnet_mape])
ax.set_xlabel('Model')
# ax.set_ylim(0.0, 0.9)
ax.set_ylabel('Mean Absolute Percentage Error (MAPE) %')

### CORR

In [ ]:
mlp_corr = corr(data_true, mlp_predicted[:, 0])
mlp_corr

In [ ]:
lstm_corr = corr(data_true, lstm_predicted)
lstm_corr

In [ ]:
gru_corr = corr(data_true, gru_predicted)
gru_corr

In [ ]:
cnn_corr = corr(data_true, cnn_predicted)
cnn_corr

In [ ]:
cnn_lstm_corr = corr(data_true, cnn_lstm_predicted[:, 0])
cnn_lstm_corr

In [ ]:
lstnet_corr = corr(data_true, lstnet_predicted[:, 0])
lstnet_corr

In [ ]:
import scipy
scipy.stats.pearsonr(data_true, lstnet_predicted[:, 0])

In [ ]:
fig = plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.bar(x=['ARIMA', 'MLP', 'LSTM', 'GRU', 'CNN', 'CNN+LSTM', 'LSTNet'], 
        height=[0.88, mlp_corr, lstm_corr, gru_corr, cnn_corr, cnn_lstm_corr, lstnet_corr])
ax.set_xlabel('Model')
ax.set_ylim(0.85, 0.95)
ax.set_ylabel('Mean Absolute Percentage Error (MAPE) %')

### r2

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
mlp_r2 = r2_score(data_true, mlp_predicted[:, 0])
lstm_r2 = r2_score(data_true, lstm_predicted)
gru_r2 = r2_score(data_true, gru_predicted)
cnn_r2 = r2_score(data_true, cnn_predicted)
cnn_lstm_r2 = r2_score(data_true, cnn_lstm_predicted[:, 0])
lstnet_r2 = r2_score(data_true, lstnet_predicted[:, 0])


In [ ]:
print(f'mlp r2:     {mlp_r2}')
print(f'lstm r2:    {lstm_r2}')
print(f'gru r2:     {gru_r2}')
print(f'cnn r2:     {cnn_r2}')
print(f'cnn+gru r2: {cnn_lstm_r2}')
print(f'lstnet r2:  {lstnet_r2}')


### RRSE

In [ ]:
mlp_rrse = rse(data_true, mlp_predicted[:, 0])
mlp_rrse

In [ ]:
lstm_rrse = rse(data_true, lstm_predicted)
lstm_rrse

In [ ]:
gru_rrse = rse(data_true, gru_predicted)
gru_rrse

In [ ]:
cnn_rrse = rse(data_true, cnn_predicted)
cnn_rrse

In [ ]:
cnn_lstm_rrse = rse(data_true, cnn_lstm_predicted[:, 0].flatten())
cnn_lstm_rrse

In [ ]:
lstnet_rrse = rse(data_true, lstnet_predicted[:, 0].flatten())
lstnet_rrse

In [ ]:
fig = plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.bar(x=['ARIMA', 'MLP', 'LSTM', 'GRU', 'CNN', 'CNN+LSTM', 'LSTNet'], 
        height=[0.79, mlp_rrse,lstm_rrse,gru_rrse,cnn_rrse,cnn_lstm_rrse,lstnet_rrse])
ax.set_xlabel('Model')
# ax.set_ylim(0.0, 0.9)
ax.set_ylabel('Root Relative Squared Error (RRSE)')

### MAE

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mlp_mae = mean_absolute_error(data_true, mlp_predicted[:, 0])
mlp_mae

In [ ]:
lstm_mae = mean_absolute_error(data_true, lstm_predicted)
lstm_mae

In [ ]:
gru_mae = mean_absolute_error(data_true, gru_predicted)
gru_mae

In [ ]:
cnn_mae = mean_absolute_error(data_true, cnn_predicted)
cnn_mae

In [ ]:
cnn_lstm_mae = mean_absolute_error(data_true, cnn_lstm_predicted[:, 0])
cnn_lstm_mae

In [ ]:
lstnet_mae = mean_absolute_error(data_true, lstnet_predicted[:, 0])
lstnet_mae

In [ ]:
fig = plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.bar(x=['ARIMA', 'MLP', 'LSTM', 'GRU', 'CNN', 'CNN+LSTM', 'LSTNet'], 
        height=[79, mlp_mae, lstm_mae, gru_mae, cnn_mae, cnn_lstm_mae, lstnet_mae])
ax.set_xlabel('Model')
# ax.set_ylim(0.0, 0.9)
ax.set_ylabel('Mean Absolute Percentage Error (MAPE) %')

### RMSRE (root mean square relative error)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mlp_mae = mean_absolute_error(data_true, mlp_predicted[:, 0])
mlp_mae

In [ ]:
lstm_mae = mean_absolute_error(data_true, lstm_predicted)
lstm_mae

In [ ]:
gru_mae = mean_absolute_error(data_true, gru_predicted)
gru_mae

In [ ]:
cnn_mae = mean_absolute_error(data_true, cnn_predicted)
cnn_mae

In [ ]:
cnn_lstm_mae = mean_absolute_error(data_true, cnn_lstm_predicted[:, 0])
cnn_lstm_mae

In [ ]:
lstnet_mae = mean_absolute_error(data_true, lstnet_predicted[:, 0])
lstnet_mae

In [ ]:
fig = plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.bar(x=['ARIMA', 'MLP', 'LSTM', 'GRU', 'CNN', 'CNN+LSTM', 'LSTNet'], 
        height=[79, mlp_mae, lstm_mae, gru_mae, cnn_mae, cnn_lstm_mae, lstnet_mae])
ax.set_xlabel('Model')
# ax.set_ylim(0.0, 0.9)
ax.set_ylabel('Mean Absolute Percentage Error (MAPE) %')

### CDF of Relative Error

In [ ]:
mlp_relative_error = (data_true - mlp_predicted[:, 0]) / data_true
lstm_relative_error = (data_true - lstm_predicted) / data_true

In [ ]:
mlp_residual_error = (data_true - mlp_predicted[:, 0])

In [ ]:
mlp_relative_error

In [ ]:
plt.plot(mlp_relative_error)

In [ ]:
num_bins = 40
counts, bin_edges = np.histogram (abs(mlp_relative_error), bins=num_bins, normed=True)
cdf = np.cumsum (counts)
plt.plot (bin_edges[1:], cdf/cdf[-1])
counts, bin_edges = np.histogram (abs(lstm_relative_error), bins=num_bins, normed=True)
cdf = np.cumsum (counts)
plt.plot (bin_edges[1:], cdf/cdf[-1])

## ARIMA

In [ ]:
arima_actual = np.load('predicted_results/arima_data_actual_multivar_1.npy')
arima_predicted = np.load('predicted_results/arima_data_predictions_multivar_1.npy')

In [ ]:
arima_actual.shape

In [ ]:
arima_predicted.shape

In [ ]:
plt.plot(arima_actual[0:300])
plt.plot(arima_predicted[0:300])

In [ ]:
arima_rrse = rse(arima_actual, arima_predicted)
arima_rrse

In [ ]:
from sklearn.metrics import mean_absolute_error
arima_mae = mean_absolute_error(arima_actual, arima_predicted)
arima_mae

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100

arima_mape = mean_absolute_percentage_error(arima_actual, arima_predicted)
arima_mape

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

arima_rmse = sqrt(mean_squared_error(arima_actual, arima_predicted))
arima_rmse

In [ ]:
arima_corr = corr(arima_actual, arima_predicted)
arima_corr

# Prediction for all models on a single route (from test dataset)

In [ ]:
test_sample = np.load('numpy_group_test_data/1_stream_test_sample_group_95.npy')
test_target = np.load('numpy_group_test_data/1_stream_test_target_group_95.npy')

normalize_mean = 9726.925995055048
normalize_std  = 6670.752950995383


In [ ]:
test_target[:,0]

In [ ]:
import statsmodels.api as sm 

arma_mod = sm.tsa.ARIMA(test_target[:,0], order=(8,0,8))
arma_res = arma_mod.fit(trend='nc', disp=-1)
arma_predictions = arma_res.predict()

In [ ]:
arma_predictions

### Load weights for models

In [ ]:
# Create a new model instance

# Restore the weights
linear.load_weights('./checkpoints/linear_weights')


In [ ]:
linear_test_predict = linear.predict(test_sample)
lstm_test_predict = lstm_model.predict(test_sample)
gru_test_predict = gru_model.predict(test_sample)
cnn_test_predict = cnn_model.predict(test_sample)
cnn_lstm_test_predict = cnn_lstm_model.predict(test_sample)
lstnet_test_predict = lstnet_model.predict(test_sample)

In [ ]:
plt.rcParams["figure.figsize"] = (10,7)
plt.plot(test_target[1:,0] * normalize_std + normalize_mean, label='actual')
plt.plot(arma_predictions[1:] * normalize_std + normalize_mean, label='arima')
plt.plot(linear_test_predict[1:] * normalize_std + normalize_mean, label='mlp')
plt.plot(lstm_test_predict[1:] * normalize_std + normalize_mean, label='lstm')
plt.plot(gru_test_predict[1:] * normalize_std + normalize_mean, label='gru')
plt.plot(cnn_test_predict[1:] * normalize_std + normalize_mean, label='cnn')
plt.plot(cnn_lstm_test_predict[1:,0] * normalize_std + normalize_mean, label='cnn+gru')
plt.plot(lstnet_test_predict[1:,0] * normalize_std + normalize_mean, label='lstnet')
plt.ylabel('Throughput (Mbits/sec)')
plt.xlabel('Day')
plt.legend(loc='best')

In [ ]:
mean_absolute_percentage_error(test_target[1:,0],)